# Mise à disposition du jeu de données stroke_dataset via une API REST

## Description du dataset

Le jeu de données utilisé provient de Kaggle : [Stroke Prediction Dataset](https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset).  
Il contient des données de patients avec différentes caractéristiques médicales et sociales, ainsi que l'information si le patient a subi un accident vasculaire cérébral (AVC) ou non.

Télécharger les données et ajouter les dans un dossier data/.

Les colonnes des données sont :  
- `id` : Identifiant unique du patient  
- `gender` : Sexe  
- `age` : Âge  
- `hypertension` : Présence d'hypertension (0 ou 1)  
- `heart_disease` : Présence de maladie cardiaque (0 ou 1)  
- `ever_married` : Statut marital  
- `work_type` : Type d'emploi  
- `Residence_type` : Urbaine ou rurale  
- `avg_glucose_level` : Moyenne du taux de glucose  
- `bmi` : Indice de masse corporelle  
- `smoking_status` : Statut tabagique  
- `stroke` : Présence d'AVC (0 ou 1)

## Projet

Vous devez exposer les données patients du jeu de données via une API REST afin que les données soit utilisables par d'autres équipes (médecins, data science, étude, etc.).

Cette API REST sera développée avec FastAPI et les spécifications sont les suivantes :
| Méthode | Endpoint                                      | Fonctionnalité                                                                                                    |
| ------- | --------------------------------------------- | ----------------------------------------------------------------------------------------------------------------- |
| `GET`   | `/patients/{id}`                              | Récupère les détails d’un patient donné (via son identifiant unique)                                              |
| `GET`   | `/patients?stroke=1&gender=Female&max_age=60` | Renvoie les patients filtrés selon plusieurs critères : AVC (oui/non), genre, âge maximal                         |
| `GET`   | `/stats/`                                     | Fournit des statistiques agrégées sur les patients (ex. : nb total de patients, âge moyen, taux d’AVC, répartition hommes/femmes, etc.) |



---

## Quelques définitions


1. Qu’est-ce qu’une API REST ?

- API signifie Application Programming Interface (Interface de Programmation d’Application). C’est un ensemble de règles et de protocoles qui permettent à des logiciels de communiquer entre eux.
- REST signifie Representational State Transfer. C’est un style architectural pour concevoir des services web.
Il en existe d'autres mais REST est celui que vous rencontrerez le plus souvent.
- Vous avez utilisé une API REST via l'API Google Books.

- A quoi sert une API REST ?

    - Permet à différentes applications de communiquer facilement, même si elles sont écrites dans des langages différents.
    - Permet d’accéder à des services distants (ex : bases de données, services web) de manière standardisée.
    - Facilite la création d’applications modulaires et évolutives (front-end, back-end, mobile, etc.)

2. Principes clés d’une API REST

- a. Utilisation du protocole HTTP
Les échanges entre client et serveur utilisent des méthodes HTTP standard comme :

    - GET : pour récupérer des données
    - POST : pour envoyer ou créer des données
    - PUT : pour mettre à jour des données
    - DELETE : pour supprimer des données

- b. Accès aux ressources via des URLs

Chaque ressource (par exemple un livre, un utilisateur) est accessible via une URL unique.

Exemple fictif:
    https://api.example.com/books/123 pour accéder au livre d’identifiant 123.

- c. Stateless (sans état)

Le serveur ne conserve aucune information sur le client entre deux requêtes. Chaque requête doit contenir toutes les informations nécessaires.

- d. Représentations des données

Les données sont envoyées et reçues généralement en format JSON ou XML, qui sont faciles à lire et à manipuler.

- e. Utilisation de codes status HTTP

Chaque réponse du serveur est accompagnée d’un code HTTP indiquant le résultat de la requête. (cf [liste des codes](https://developer.mozilla.org/en-US/docs/Web/HTTP/Reference/Status))

## Outils utilisés
1. FastAPI

FastAPI est un framework Python moderne, rapide et très utilisé dans le milieu professionnel pour construire des API REST. 

Il permet :
- une définition simple des routes et des paramètres  
- La génération automatique de documentation interactive (Swagger UI)  
- FastAPI lit les requêtes entrantes, les traite avec ton code Python, et retourne une réponse HTTP (en JSON).

2. Uvicorn : exécute l'application FastAPI

- Uvicorn est un serveur ASGI (Asynchronous Server Gateway Interface) : c'est une interface standard pour gérer les requêtes de manière asynchrone et performante, notamment utile pour les applications modernes.
- Il attend les requêtes HTTP (par exemple depuis un navigateur), les transmet à FastAPI, et renvoie la réponse.
- Uvicorn permet à l'API de fonctionner : sans Uvicorn ou un autre serveur, FastAPI ne peut pas fonctionner.


3. Swagger UI : l’interface de doc et test interactive

- Swagger UI est généré automatiquement par FastAPI.
- C’est une interface web qui permet de :
    - Voir toutes les routes disponibles dans l'API
    - Tester les routes en envoyant des requêtes sans écrire de code (bouton try it out)
    - Voir les paramètres attendus et les formats de réponse
    
4. Résumé des interactions

- Tester la route de base de l'API grâce à la commande :
```bash
    poetry run fastapi dev stroke_api/main.py
```

--> Qu'est-ce qu'il se passe derrière cette commande ?

- Uvicorn démarre un serveur local
- FastAPI génère automatiquement une interface : Swagger UI, accessible sur http://127.0.0.1:8000/docs qui affiche toutes les routes définies dans le code python FastAPI
- Quand on clique sur "Try it out" dans Swagger UI, Swagger envoie une requête HTTP au serveur (ici Uvicorn)
- Le serveur (Uvicorn) la reçoit, l’envoie à FastAPI, qui traite et renvoie une réponse
- Swagger UI affiche la réponse de l’API (par ex : liste de patients)

---

Import des bibliothèques utiles au projet

In [1]:
import pandas as pd
import numpy as np

## 1. Prétraitement des données / Data preprocessing

Les données réelles sont rarement prêtes à être utilisées directement. Elles peuvent contenir des erreurs, des valeurs manquantes, des doublons, des formats incohérents, ou ne pas être adaptées au modèle ou au système cible.

Le prétraitement consiste à nettoyer, structurer et transformer les données brutes avant de les exploiter dans un projet (modèle IA, API, visualisation, etc.).

Vous avez déjà prétraité des données, petit rappel des éléments sur lesquels travailler dans un prétraitment classique et les méthodes pandas qu'il est possible d'utiliser pour les différentes étapes (des exeples d'utilisation des méthodes pandas sont disponibles dans la doc) : 
- explorer les données pour identifier les types de données, valeurs manquantes, incohérence ([info](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.info.html), [dtypes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dtypes.html))
- adapter les types si nécessaire ([astypes](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html))
- identifier les doublons et les supprimer s'il y en a ([duplicated](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html), [drop_duplicates](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html))
- traiter les valeurs manquantes s'il y en a ([fillna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html), [dropna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html), [replace](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html))
- identifier les incohérences éventuelles (valeurs aberrantes/outliers) en vérifiant si les valeurs min, max, moyennes sont raisonnables (recherche internet si nécessaire) ([describe](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html)), et les traiter.
- Traiter les valeurs aberrantes si vous en détectez ([loc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html) pour récupérer les lignes qui répondent à une certaine condition, cf exemple ci-dessous)


**Exemple df.loc :**

Récupérer toutes les lignes de df telles que la valeur de "nom de colonne" >= 0

```df_subset = df.loc[stroke_data_df['nom de colonne'] >= 0]```



---
### **TODO**
1.a. Prétraiter les données du dataset.


1.b. Documenter dans le README.md :
- Les étapes de prétraitement,
- Justification des choix concernant le traitement des valeurs manquantes (si besoin),
- Liste des valeurs raisonnables utilisées pour détecter les valeurs aberrantes, 
- Justification des choix pour traiter les valeurs aberrantes (si besoin).

2.a. Chercher des infos sur le format de fichier parquet et indiquer les sources consultées : 
- https://www.datacamp.com/fr/tutorial/apache-parquet  

- https://www.icem7.fr/cartographie/parquet-devrait-remplacer-le-format-csv/  

- https://pythonds.linogaliana.fr/content/manipulation/05_parquet_s3.html#:~:text=Le%20format%20Parquet%20est%20un,pro%C3%A9minents%20sont%20Arrow%20et%20DuckDB%20  

- Différence principale avec le format csv ?  
    - Le format csv est plus lourd (non compressé). Parquet est orienté colonne là où csv est orienté lignes, ce qui permet de charger uniquement les colonnes utiles à l'analyse. Il conserve les types de données (float, int, etc.).

- Dans quels cas l'utiliser ?  
    - Pour des traitements Big Data, besoin de performance, données volumineuses.

- Pourquoi c'est un format adapté aux gros volumes de données ?
    - Stockage par colonne = accès plus rapide + moins de données à charger.
    - Très bonne compression = gain d’espace disque.
    - Lecture sélective = gain de performance.
    - Compatible avec outils distribués = scalabilité.



2.b. Sauvegarder les données prétraiteées dans un fichier parquet ([to_parquet](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_parquet.html)).


### Prétraitement des données

In [2]:
# Importation des données
try:
	df = pd.read_csv('../data/healthcare-dataset-stroke-data.csv')
	print(df.head())
except FileNotFoundError: # Gestion de l'erreur si le fichier n'est pas trouvé
	print("Le fichier '../data/healthcare-dataset-stroke-data.csv' est introuvable. Veuillez le télécharger depuis Kaggle et le placer dans le dossier 'data/'.")

      id  gender   age  hypertension  heart_disease ever_married  \
0   9046    Male  67.0             0              1          Yes   
1  51676  Female  61.0             0              0          Yes   
2  31112    Male  80.0             0              1          Yes   
3  60182  Female  49.0             0              0          Yes   
4   1665  Female  79.0             1              0          Yes   

       work_type Residence_type  avg_glucose_level   bmi   smoking_status  \
0        Private          Urban             228.69  36.6  formerly smoked   
1  Self-employed          Rural             202.21   NaN     never smoked   
2        Private          Rural             105.92  32.5     never smoked   
3        Private          Urban             171.23  34.4           smokes   
4  Self-employed          Rural             174.12  24.0     never smoked   

   stroke  
0       1  
1       1  
2       1  
3       1  
4       1  


In [3]:
# On affiche les informations du dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


In [4]:
# On vérifie s'il y a des doublons dans le dataframe et on les supprimes
df_duplicated = df.duplicated()
print(f"Nombre de doublons : {df_duplicated.sum()}")

df = df.drop_duplicates()

Nombre de doublons : 0


In [5]:
# On calcul le nombre de valeurs manquantes dans la colonne 'bmi'
df_bmi = df['bmi'].isnull().sum()
print(f"Nombre de valeurs manquantes dans la colonne 'bmi' : {df_bmi}")

Nombre de valeurs manquantes dans la colonne 'bmi' : 201


In [6]:
# Fonction pour détecter les valeurs aberrantes dans le dataframe
def aberrante_values(df):
    """
    Fonction pour détecter les valeurs aberrantes dans le dataframe.
    Cette fonction vérifie si la colonne 'work_type' est présente et si l'âge est inférieur à 18 ans.
    Si c'est le cas, elle remplace la valeur dans 'work_type' par 'children' si l'âge est inférieur à 18 ans et que 'work_type' n'est pas 'children'.
    On vérifie et traite également les valeurs 'Unknown' dans la colonne 'smoking_status' selon l'âge < ou > 18 ans. 
    
    Args:
        df (DataFrame): Le dataframe à analyser.
    
    Returns:
        DataFrame: Le dataframe avec les valeurs aberrantes traitées."""
    
    if 'work_type' in df.columns and 'age' in df.columns:
        df.loc[(df['age'] < 18) & (df['work_type'] != 'children'), 'work_type'] = 'children'
    
    if 'smoking_status' in df.columns and 'age' in df.columns:
        df.loc[(df['age'] < 18) & (df['smoking_status'] == 'Unknown'), 'smoking_status'] = 'never smoked'
        df.loc[(df['age'] >= 18) & (df['smoking_status'] == 'Unknown'), 'smoking_status'] = 'not specified'
    
    return df

df = aberrante_values(df)

In [7]:
# On calcule et affiche la mediane pour 'bmi'
print(df.groupby(['gender', 'age', 'Residence_type', 'work_type'])['bmi'].median())


gender  age    Residence_type  work_type    
Female  0.08   Urban           children         14.1
        0.32   Rural           children         16.1
               Urban           children         19.6
        0.40   Rural           children         17.4
        0.48   Rural           children         16.1
                                                ... 
Male    82.00  Rural           Self-employed    27.0
               Urban           Govt_job         29.0
                               Private          28.5
                               Self-employed    24.3
Other   26.00  Rural           Private          22.4
Name: bmi, Length: 833, dtype: float64


In [8]:
# On applique la mediane pour les NaN dans 'bmi'
df['bmi'] = df.groupby(['gender', 'age', 'Residence_type', 'work_type'])['bmi'].transform(lambda x: round(x.fillna(x.median()), 1))
print(df['bmi'])

bmi = df['bmi'].isnull().sum()
print(f"----------------{bmi}----------------")

0       36.6
1       26.8
2       32.5
3       34.4
4       24.0
        ... 
5105    26.9
5106    40.0
5107    30.6
5108    25.6
5109    26.2
Name: bmi, Length: 5110, dtype: float64
----------------9----------------


/home/loic/.cache/pypoetry/virtualenvs/stroke-api-AzSXMV3o-py3.12/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/loic/.cache/pypoetry/virtualenvs/stroke-api-AzSXMV3o-py3.12/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/loic/.cache/pypoetry/virtualenvs/stroke-api-AzSXMV3o-py3.12/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/loic/.cache/pypoetry/virtualenvs/stroke-api-AzSXMV3o-py3.12/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/loic/.cache/pypoetry/virtualenvs/stroke-api-AzSXMV3o-py3.12/lib/python3.12/site-packages/n

In [9]:
# On applique une autre mediane moins restrictive pour les valeurs qui n'ont pas reçu la précédente mediane'bmi'
df['bmi'] = df.groupby(['gender', 'work_type'])['bmi'].transform(lambda x: round(x.fillna(x.median()), 1))
print(df['bmi'])

bmi = df['bmi'].isnull().sum()
print(f"----------------{bmi}----------------")

0       36.6
1       26.8
2       32.5
3       34.4
4       24.0
        ... 
5105    26.9
5106    40.0
5107    30.6
5108    25.6
5109    26.2
Name: bmi, Length: 5110, dtype: float64
----------------0----------------


In [10]:
# Vérification valeurs aberrantes sur le glucose
if 'avg_glucose_level' in df.columns:
    df_glucose = df.loc[(df['avg_glucose_level'] < 50) | (df['avg_glucose_level'] > 280)]
    print(df_glucose)

Empty DataFrame
Columns: [id, gender, age, hypertension, heart_disease, ever_married, work_type, Residence_type, avg_glucose_level, bmi, smoking_status, stroke]
Index: []


In [11]:
# Vérification valeurs aberrantes sur le bmi
if 'bmi' in df.columns:
    df_bmi = df.loc[(df['bmi'] < 10) | (df['bmi'] > 80)]
    print(df_bmi)

         id gender   age  hypertension  heart_disease ever_married work_type  \
2128  56420   Male  17.0             1              0           No  children   
4209  51856   Male  38.0             1              0          Yes   Private   

     Residence_type  avg_glucose_level   bmi smoking_status  stroke  
2128          Rural              61.67  97.6   never smoked       0  
4209          Rural              56.90  92.0   never smoked       0  


In [12]:
# Sauvegarde du df dans un fichier .parquet
df.to_parquet('../data/stroke_data.parquet')

-----
## Développement de l'API

A présent que les données sont propres, on peut débuter la création de l'API.

Pour cela, vous allez avoir besoin de quelques fonctions permettant de filtrer les données.

Vous allez les définir ci-dessous, ce qui vous permettra de les tester puis les fonctions seront reportées dans le fichier filters.py.

## Route `/patients/`
- Cette route retourne une liste filtrée de patients
- On souhaite pouvoir filtrer par `gender`, `stroke` ou `max_age`

L'objectif est ici de définir une fonction python qui prend en entrée les paramètres optionnels : _gender_, *stroke*, *max_age* et qui renvoie un dictionnaire filtré des données.

On décompose la rédaction de cette fonction en plusieurs étapes. 

Dans un premier temps, écrire et tester les filtres que l'on souhaite appliquer sur les données (utiliser [loc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html))

In [13]:
# Filtrer le dataframe pour ne garder que les patients pour lesquels "stroke=1"
def filtred_stroke(df, stroke):
    if 'stroke' in df.columns:
        return df.loc[df['stroke'] == stroke]
    else:
        return df

df_stroke = filtred_stroke(df, 1)
print(df_stroke)



        id  gender   age  hypertension  heart_disease ever_married  \
0     9046    Male  67.0             0              1          Yes   
1    51676  Female  61.0             0              0          Yes   
2    31112    Male  80.0             0              1          Yes   
3    60182  Female  49.0             0              0          Yes   
4     1665  Female  79.0             1              0          Yes   
..     ...     ...   ...           ...            ...          ...   
244  17739    Male  57.0             0              0          Yes   
245  49669  Female  14.0             0              0           No   
246  27153  Female  75.0             0              0          Yes   
247  34060    Male  71.0             1              0          Yes   
248  43424  Female  78.0             0              0          Yes   

         work_type Residence_type  avg_glucose_level   bmi   smoking_status  \
0          Private          Urban             228.69  36.6  formerly smoked   
1

In [35]:
# Filtrer les données pour ne garder que les patients pour lesquels "gender="male"
def filtred_gender(df, gender):
    if 'gender' in df.columns:
        return df.loc[df['gender'].str.lower() == gender.lower()]
    else:
        return df

df_gender = filtred_gender(df, 'male')
print(df_gender)


         id gender   age  hypertension  heart_disease ever_married  \
0      9046   Male  67.0             0              1          Yes   
2     31112   Male  80.0             0              1          Yes   
5     56669   Male  81.0             0              0          Yes   
6     53882   Male  74.0             1              1          Yes   
13     8213   Male  78.0             0              1          Yes   
...     ...    ...   ...           ...            ...          ...   
5097  64520   Male  68.0             0              0          Yes   
5098    579   Male   9.0             0              0           No   
5099   7293   Male  40.0             0              0          Yes   
5100  68398   Male  82.0             1              0          Yes   
5108  37544   Male  51.0             0              0          Yes   

          work_type Residence_type  avg_glucose_level   bmi   smoking_status  \
0           Private          Urban             228.69  36.6  formerly smoked   

In [15]:
# Filtrer les données pour ne garder que les patients tels que "age <= max_age"
def filtred_max_age(df, max_age):
    if 'age' in df.columns:
        return df.loc[df['age'] <= max_age]
    else:
        return df 

df_max_age = filtred_max_age(df, max_age=50)
print(df_max_age)


         id  gender   age  hypertension  heart_disease ever_married  \
3     60182  Female  49.0             0              0          Yes   
15    58202  Female  50.0             1              0          Yes   
31    33879    Male  42.0             0              0          Yes   
34    14248    Male  48.0             0              0           No   
39    62602  Female  49.0             0              0          Yes   
...     ...     ...   ...           ...            ...          ...   
5101  36901  Female  45.0             0              0          Yes   
5103  22127  Female  18.0             0              0           No   
5104  14180  Female  13.0             0              0           No   
5107  19723  Female  35.0             0              0          Yes   
5109  44679  Female  44.0             0              0          Yes   

          work_type Residence_type  avg_glucose_level   bmi smoking_status  \
3           Private          Urban             171.23  34.4         s

Appliquer successivement les 3 filtres au sein d'une fonction qui prend en entrée le dataframe, _stroke_, _gender_, _max_age_ et qui renvoie une liste de dictionnaire de patients (utiliser la méthode pandas [to_dict](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_dict.html)).

Exemple
```
[{'id': 9046,
  'gender': 'Male',
  'age': 67.0,
  ...
  'smoking_status': 'formerly smoked',
  'stroke': 1},
 {'id': 31112,
  'gender': 'Male',
  'age': 80.0,
  ...
  'smoking_status': 'formerly smoked',
  'stroke': 1}]
  ```

In [37]:
def filter_patient(df, max_age, gender, stroke):
    df_filtered = filtred_stroke(df, stroke)
    df_filtered = filtred_gender(df_filtered, gender)
    df_filtered = filtred_max_age(df_filtered, max_age)
    return df_filtered

df_result = filter_patient(df, stroke = 1, gender='male', max_age=50)

df_result.to_dict('records')


[{'id': 33879,
  'gender': 'Male',
  'age': 42.0,
  'hypertension': 0,
  'heart_disease': 0,
  'ever_married': 'Yes',
  'work_type': 'Private',
  'Residence_type': 'Rural',
  'avg_glucose_level': 83.41,
  'bmi': 25.4,
  'smoking_status': 'not specified',
  'stroke': 1},
 {'id': 14248,
  'gender': 'Male',
  'age': 48.0,
  'hypertension': 0,
  'heart_disease': 0,
  'ever_married': 'No',
  'work_type': 'Govt_job',
  'Residence_type': 'Urban',
  'avg_glucose_level': 84.2,
  'bmi': 29.7,
  'smoking_status': 'never smoked',
  'stroke': 1},
 {'id': 42117,
  'gender': 'Male',
  'age': 43.0,
  'hypertension': 0,
  'heart_disease': 0,
  'ever_married': 'Yes',
  'work_type': 'Self-employed',
  'Residence_type': 'Urban',
  'avg_glucose_level': 143.43,
  'bmi': 45.9,
  'smoking_status': 'not specified',
  'stroke': 1},
 {'id': 14499,
  'gender': 'Male',
  'age': 47.0,
  'hypertension': 0,
  'heart_disease': 0,
  'ever_married': 'Yes',
  'work_type': 'Private',
  'Residence_type': 'Urban',
  'avg_gl

A présent on souhaite ajouter des informations sur les types des paramètres et valeurs de retour de la fonction pour faciliter sa compréhension et son utilisation, ce qu’on appelle l’annotation de type (type hinting).

Cette pratique facilite la lecture et la maintenance du code.

Quels changements pour la fonction ?

A la suite de chaque paramètre, on ajoute le type attendu pour le paramètre. À la suite des paramètres on ajoute le type de ce que qui est retourné par la fonction, dans l'exemple ici : 

```def filter_patient(stroke_data_df: pd.DataFrame, gender: str, etc) -> list[dict]```

Ajouter les types dans la définition de la fonction.

Tester la fonction en ne mettant pas de valeur pour *max_age*.

Que se passe-t-il ?  
On a une erreur de type TypeError : Un argument est attendu pour max_age

In [17]:
def filter_patient(df: pd.DataFrame, gender: str, stroke: int, max_age: int) -> list[dict]:
    df_filtered = filtred_stroke(df, stroke)
    df_filtered = filtred_gender(df_filtered, gender)
    df_filtered = filtred_max_age(df_filtered, max_age)
    
    return df_filtered

df_result = filter_patient(df, stroke = 1, gender='Male')

df_result.to_dict('records')

TypeError: filter_patient() missing 1 required positional argument: 'max_age'

Dans la fonction écrite ci-dessus, chaque paramètre est obligatoire. 

On souhaite pouvoir filtrer les patients sur 0, 1 ou 2 des paramètres de la fonction (filtrer seulement sur *max_age*  mais ne pas appliquer de filtres sur _gender_ et _stroke_ par exemple).

On peut rendre optionnel les paramètres d'un fonction en choisissant une valeur par défault. Si on utilise la fonction en n'utilisant pas ces paramètres alors la valeur par défault est utilisé.

Copier coller votre fonction ci-dessous et ajouter en paramètre : `max_age=None`

et ajouter la condition suivante **avant le filtre** sur `max_age` : 

```if max_age is not None : ``` 

Si la fonction _filter_patient_ est appelée sans argument *max_age*, alors le filtre sur *max_age* n'est pas appliqué. 

Il est tout à fait possible de définir une valeur par défault par exemple 30 ans : dans ce cas si la fonction est appelée sans argument *max_age*, alors par défault on filtre les patients ayant moins de 30 ans.

**ATTENTION :** Les paramètres optionnels doivent toujours être à la fin de la liste de paramètres.

In [43]:
# fonction filter_patient paramètre max_age optionnel
from typing import Optional
def filter_patient(df: pd.DataFrame, gender: str, stroke: int, max_age: Optional[int] = None) -> list[dict]:
    df_filtered = df.copy()
    
    df_filtered = filtred_stroke(df, stroke)
    df_filtered = filtred_gender(df_filtered, gender)
    
    if max_age is not None:
        df_filtered = filtred_max_age(df_filtered, max_age)
    
    return df_filtered

df_result = filter_patient(df, stroke = 1, gender='male', max_age = 50)

df_result.to_dict('records')


[{'id': 33879,
  'gender': 'Male',
  'age': 42.0,
  'hypertension': 0,
  'heart_disease': 0,
  'ever_married': 'Yes',
  'work_type': 'Private',
  'Residence_type': 'Rural',
  'avg_glucose_level': 83.41,
  'bmi': 25.4,
  'smoking_status': 'not specified',
  'stroke': 1},
 {'id': 14248,
  'gender': 'Male',
  'age': 48.0,
  'hypertension': 0,
  'heart_disease': 0,
  'ever_married': 'No',
  'work_type': 'Govt_job',
  'Residence_type': 'Urban',
  'avg_glucose_level': 84.2,
  'bmi': 29.7,
  'smoking_status': 'never smoked',
  'stroke': 1},
 {'id': 42117,
  'gender': 'Male',
  'age': 43.0,
  'hypertension': 0,
  'heart_disease': 0,
  'ever_married': 'Yes',
  'work_type': 'Self-employed',
  'Residence_type': 'Urban',
  'avg_glucose_level': 143.43,
  'bmi': 45.9,
  'smoking_status': 'not specified',
  'stroke': 1},
 {'id': 14499,
  'gender': 'Male',
  'age': 47.0,
  'hypertension': 0,
  'heart_disease': 0,
  'ever_married': 'Yes',
  'work_type': 'Private',
  'Residence_type': 'Urban',
  'avg_gl

In [44]:
# test fonction sans argument max_age
from typing import Optional
def filter_patient(df: pd.DataFrame, gender: str, stroke: int, max_age: Optional[int] = None) -> list[dict]:
    df_filtered = df.copy()
    
    df_filtered = filtred_stroke(df, stroke)
    df_filtered = filtred_gender(df_filtered, gender)
    
    if max_age is not None:
        df_filtered = filtred_max_age(df_filtered, max_age)
    
    return df_filtered

df_result = filter_patient(df, stroke = 1, gender='male')

df_result.to_dict('records')


[{'id': 9046,
  'gender': 'Male',
  'age': 67.0,
  'hypertension': 0,
  'heart_disease': 1,
  'ever_married': 'Yes',
  'work_type': 'Private',
  'Residence_type': 'Urban',
  'avg_glucose_level': 228.69,
  'bmi': 36.6,
  'smoking_status': 'formerly smoked',
  'stroke': 1},
 {'id': 31112,
  'gender': 'Male',
  'age': 80.0,
  'hypertension': 0,
  'heart_disease': 1,
  'ever_married': 'Yes',
  'work_type': 'Private',
  'Residence_type': 'Rural',
  'avg_glucose_level': 105.92,
  'bmi': 32.5,
  'smoking_status': 'never smoked',
  'stroke': 1},
 {'id': 56669,
  'gender': 'Male',
  'age': 81.0,
  'hypertension': 0,
  'heart_disease': 0,
  'ever_married': 'Yes',
  'work_type': 'Private',
  'Residence_type': 'Urban',
  'avg_glucose_level': 186.21,
  'bmi': 29.0,
  'smoking_status': 'formerly smoked',
  'stroke': 1},
 {'id': 53882,
  'gender': 'Male',
  'age': 74.0,
  'hypertension': 1,
  'heart_disease': 1,
  'ever_married': 'Yes',
  'work_type': 'Private',
  'Residence_type': 'Rural',
  'avg_gl

Ajouter des valeurs par défault et les conditions pour chaque filtre.

Pour les types, on indique qu'il s'agit de paramètres optionels en utilisant le module python _typing_

```
from typing import Optional
def filter_patient(stroke_data_df: pd.DataFrame, gender: Optional[str] = None,etc)
```

Adapter les types en utilisant ce modèle.

In [65]:
# fonction avec ajout de paramètres par défault et de type
def filter_patient(
    df: pd.DataFrame, 
    gender: Optional[str] = None, 
    stroke: Optional[int] = None, 
    max_age: Optional[int] = None
) -> list[dict]:
    
    df_filtered = df.copy()

    if gender is not None:
        df_filtered = filtred_gender(df_filtered, gender)
    if stroke is not None:
        df_filtered = filtred_stroke(df_filtered, stroke)
    if max_age is not None:
        df_filtered = filtred_max_age(df_filtered, max_age)
    if df_filtered.empty:
        print("Aucun patient ne correspond aux critères sélectionnés.")
    return df_filtered.to_dict('records')


filter_patient(df, stroke = 1, gender = 'female', max_age = 50)



[{'id': 60182,
  'gender': 'Female',
  'age': 49.0,
  'hypertension': 0,
  'heart_disease': 0,
  'ever_married': 'Yes',
  'work_type': 'Private',
  'Residence_type': 'Urban',
  'avg_glucose_level': 171.23,
  'bmi': 34.4,
  'smoking_status': 'smokes',
  'stroke': 1},
 {'id': 58202,
  'gender': 'Female',
  'age': 50.0,
  'hypertension': 1,
  'heart_disease': 0,
  'ever_married': 'Yes',
  'work_type': 'Self-employed',
  'Residence_type': 'Rural',
  'avg_glucose_level': 167.41,
  'bmi': 30.9,
  'smoking_status': 'never smoked',
  'stroke': 1},
 {'id': 62602,
  'gender': 'Female',
  'age': 49.0,
  'hypertension': 0,
  'heart_disease': 0,
  'ever_married': 'Yes',
  'work_type': 'Private',
  'Residence_type': 'Urban',
  'avg_glucose_level': 60.91,
  'bmi': 29.9,
  'smoking_status': 'never smoked',
  'stroke': 1},
 {'id': 36338,
  'gender': 'Female',
  'age': 39.0,
  'hypertension': 1,
  'heart_disease': 0,
  'ever_married': 'Yes',
  'work_type': 'Private',
  'Residence_type': 'Rural',
  'avg_

Tester la fonction sans argument pour les filtres, elle doit donc renvoyer le dataframe non filtré.

In [66]:
# test fonction sans argument pour les filtres
filter_patient(df)

[{'id': 9046,
  'gender': 'Male',
  'age': 67.0,
  'hypertension': 0,
  'heart_disease': 1,
  'ever_married': 'Yes',
  'work_type': 'Private',
  'Residence_type': 'Urban',
  'avg_glucose_level': 228.69,
  'bmi': 36.6,
  'smoking_status': 'formerly smoked',
  'stroke': 1},
 {'id': 51676,
  'gender': 'Female',
  'age': 61.0,
  'hypertension': 0,
  'heart_disease': 0,
  'ever_married': 'Yes',
  'work_type': 'Self-employed',
  'Residence_type': 'Rural',
  'avg_glucose_level': 202.21,
  'bmi': 26.8,
  'smoking_status': 'never smoked',
  'stroke': 1},
 {'id': 31112,
  'gender': 'Male',
  'age': 80.0,
  'hypertension': 0,
  'heart_disease': 1,
  'ever_married': 'Yes',
  'work_type': 'Private',
  'Residence_type': 'Rural',
  'avg_glucose_level': 105.92,
  'bmi': 32.5,
  'smoking_status': 'never smoked',
  'stroke': 1},
 {'id': 60182,
  'gender': 'Female',
  'age': 49.0,
  'hypertension': 0,
  'heart_disease': 0,
  'ever_married': 'Yes',
  'work_type': 'Private',
  'Residence_type': 'Urban',
  

Cette fonction va être utilisée dans la définition de l'API pour créer une route qui permette d'accéder à des données filtrées sur les patients.

Dans le fichier de définition de l'API, toutes les fonctions vont travailler sur les données du fichier. 

Pour alléger les fonctions on va donc utiliser une **variable globale** pour les données et supprimer le paramètre `df` de la fonction.

On lit les données en début de fichier puis on travaille au sein des fonctions sur une copie du dataframe de données.


**En résumé les modifications à faire sont :**


- Supprimer le paramètre df de la fonction,
- Ajouter en début de fonction :  
```df = stroke_data_df.copy()```

1. Dans le fichier filters.py, il suffit d'ajouter : 
- lecture du fichier de données prétraitée dans la variable *df* en début de fichier (utiliser pandas),
- @app.get("/patients/") pour définir le route,
puis la fonction.

2. Dans le fichier api.py: appeler la fonction dans la route correspondante.

Tester la route avec 

```poetry run fastapi dev stroke_api/main.py```

http://127.0.0.1:8000/docs : utiliser la fonctionnalité Try it out pour tester la route.

---
## Autres routes

De la même manière, créer les fonctions appropriées pour la création de :
- la route `/patients/{id}` : Récupère les détails d’un patient donné (via son identifiant unique) 

- la route `/stats/` : Fournit des statistiques agrégées sur les patients (ex. : nb total de patients, âge moyen, taux d’AVC, répartition hommes/femmes).

- Lister les tâches à faire sous forme d'issue github : travailler sur une branche différentes pour l'ajout de chacune des routes.